In [ ]:
import numpy as np

import tensorflow as tf
import keras
from preprocess import preprocess


print(tf.config.list_physical_devices('GPU'))

In [ ]:
# hehe funny number
np.random.seed(69)

print(np.random.rand(1,1))

# I'm not a patient man, if training saturated we can stop early
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# choose the dataset you want to use, cwru, mfd or tri
DATASET = 'mfd'
PERCENT_SAMPLE = 0.01 # choose the percentage of the dataset you want to use for training, 0.05 is 5%

xtrain, ytrain, xtest, ytest, classes, window_size = preprocess(DATASET)

# it wouldn't be transfer learning if I didn't sample the data into a smaller dataset so my experiment has some validity
indices = np.random.choice(xtrain.shape[0], size=int(len(xtrain)*PERCENT_SAMPLE), replace=False)
xtrain = xtrain[indices]
ytrain = ytrain[indices]

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=(window_size,1)))

model.add(keras.layers.Conv1D(filters=128, kernel_size=9, activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))

model.add(keras.layers.Conv1D(filters=64, kernel_size=9, activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))

model.add(keras.layers.Conv1D(filters=32, kernel_size=9, activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(len(classes)))
model.add(keras.layers.Softmax())

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# hehe funny number
np.random.seed(69)

print(np.random.rand(1,1))

# I'm not a patient man, if training saturated we can stop early
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

DATASET = 'mfd' # choose the dataset you want to use, cwru, mfd or tri
MODEL = 'cwru' # choose the model you want to use, cwru, mfd or tri
PERCENT_SAMPLE = 0.01 # choose the percentage of the dataset you want to use for training, 0.05 is 5%

pretrained_model = keras.models.load_model(f'models/{MODEL}_model.h5')

transfer_model = keras.models.Sequential()
transfer_model.add(keras.layers.Input(shape=(window_size,1)))

for layer in pretrained_model.layers[:-7]:
    layer.trainable = False
    transfer_model.add(layer)

for layer in transfer_model.layers[0:2]:
    layer.trainable = True

transfer_model.add(keras.layers.Flatten())

transfer_model.add(keras.layers.Dense(32, activation='relu'))
transfer_model.add(keras.layers.Dropout(0.4))

transfer_model.add(keras.layers.Dense(16, activation='relu'))
transfer_model.add(keras.layers.Dropout(0.2))

transfer_model.add(keras.layers.Dense(len(classes)))
transfer_model.add(keras.layers.Softmax())

opt = keras.optimizers.Adam(learning_rate=0.001)
transfer_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
wcyd = []
for i in range(20):
    model.fit(xtrain, ytrain, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    _, validation_accuracy = model.evaluate(xtest, ytest, verbose=0)

    transfer_model.fit(xtrain, ytrain, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    _, validation_accuracy_tf = transfer_model.evaluate(xtest, ytest, verbose=0)

    wcyd.append((validation_accuracy*100, validation_accuracy_tf*100))
    print(f'{i} training completed')

In [ ]:
differences = [round(j-i,2) for i,j in wcyd]

In [ ]:
mean_diff = sum(differences)/len(differences)
mean_acc = sum([j for i,j in wcyd])/len(wcyd)

print(f'Mean difference: {mean_diff:.2f}')
print(f'Mean accuracy: {mean_acc:.2f}')